In [2]:
from llama_cpp import Llama
from llama_cpp_agent.providers import LlamaCppPythonProvider

# Create an instance of the Llama class and load the model
llama_model = Llama("/Users/benedikt/.cache/lm-studio/models/NousResearch/Hermes-3-Llama-3.1-8B-GGUF/Hermes-3-Llama-3.1-8B.Q4_K_M.gguf", n_batch=1024, n_threads=10, n_gpu_layers=40, n_ctx=10000)

# Create the provider by passing the Llama class instance to the LlamaCppPythonProvider class
provider = LlamaCppPythonProvider(llama_model)

Hermes-3-Llama-3.1-8B.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 27 key-value pairs and 292 tensors from /home/bkantz/.cache/huggingface/hub/models--NousResearch--Hermes-3-Llama-3.1-8B-GGUF/snapshots/307a5dfb59aa38d88b6cfd32f44b8ad7c1da9fb8/./Hermes-3-Llama-3.1-8B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Hermes 3 Llama 3.1 8B
llama_model_loader: - kv   3:                       general.organization str              = NousResearch
llama_model_loader: - kv   4:                           general.basename str              = Hermes-3-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B


NameError: name 'LlamaCppPythonProvider' is not defined

In [13]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))
sys.path.append(os.path.abspath("../backend"))
sys.path.append(os.path.abspath(""))

In [14]:
from backend.ontology import *
base_path = "../data"
onto_path = base_path + "/hero-ontology/hereditary_clinical.ttl"
brainteaser_graph = Graph().parse(onto_path, format="turtle")
brainteaser_graph.bind("bto", "http://www.semanticweb.org/ontologies/2020/3/bto#")
oman = OntologyManager(OntologyConfig(), brainteaser_graph)

In [15]:
classes=oman.q_to_df("""
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?s
WHERE {
    ?s rdf:type owl:Class.
}
""")[0].to_list()
classes

[rdflib.term.BNode('nd0a8335c002a44b7992544d6a0d8eba9b1'),
 rdflib.term.URIRef('http://ontology.eil.utoronto.ca/GCI/Environment/Pollution.owl#Air_pollution_concentration'),
 rdflib.term.URIRef('http://purl.oclc.org/NET/ssnx/ssn#SensingDevice'),
 rdflib.term.URIRef('http://www.w3.org/2000/10/swap/pim/contact#Person'),
 rdflib.term.URIRef('http://www.w3.org/2006/time#Instant'),
 rdflib.term.URIRef('http://www.wurvoc.org/vocabularies/om-1.8/Unit_of_measure'),
 rdflib.term.URIRef('https://w3id.org/brainteaser/ontology/schema/ALSFRS'),
 rdflib.term.URIRef('https://w3id.org/brainteaser/ontology/schema/Activity'),
 rdflib.term.URIRef('https://w3id.org/brainteaser/ontology/schema/Administration'),
 rdflib.term.URIRef('https://w3id.org/brainteaser/ontology/schema/AdverseDrugReaction'),
 rdflib.term.URIRef('https://w3id.org/brainteaser/ontology/schema/AnatomicalSite'),
 rdflib.term.URIRef('https://w3id.org/brainteaser/ontology/schema/BeforeOnset'),
 rdflib.term.URIRef('https://w3id.org/brainteas

In [16]:
classes_enriched = [oman.enrich_subject(c) for  c in classes]

In [17]:
classes_enriched

[Subject(subject_id='_:nd0a8335c002a44b7992544d6a0d8eba9b1', label='Corticotropin', spos={'dcterms:conformsTo': ['https://uts.nlm.nih.gov/uts/umls/concept/C0039730', 'https://uts.nlm.nih.gov/uts/umls/concept/C0040128', 'https://uts.nlm.nih.gov/uts/umls/concept/C0025644', 'https://uts.nlm.nih.gov/uts/umls/concept/C3661975', 'https://uts.nlm.nih.gov/uts/umls/concept/C3653309', 'https://uts.nlm.nih.gov/uts/umls/concept/C0013595', 'https://uts.nlm.nih.gov/uts/umls/concept/C0002092', 'https://uts.nlm.nih.gov/uts/umls/concept/C3653568', 'https://uts.nlm.nih.gov/uts/umls/concept/C1874314', 'https://uts.nlm.nih.gov/uts/umls/concept/C0003811', 'https://uts.nlm.nih.gov/uts/umls/concept/C3653100', 'https://uts.nlm.nih.gov/uts/umls/concept/C0031212', 'https://uts.nlm.nih.gov/uts/umls/concept/C0003299', 'https://uts.nlm.nih.gov/uts/umls/concept/C0037088', 'https://uts.nlm.nih.gov/uts/umls/concept/C0162316', 'https://uts.nlm.nih.gov/uts/umls/concept/C0031412', 'https://uts.nlm.nih.gov/uts/umls/conce

In [18]:
ontology_context=""
for c in classes_enriched:
    if c.label.startswith("<"):
        continue
    ontology_context += c.label
    
    if "rdfs:subClassOf" in c.spos:
        subcls = oman.enrich_subject(c.spos["rdfs:subClassOf"][0])
        if subcls.label.startswith("<"):
            continue
        ontology_context +=" is subclass of " + subcls.label
    ontology_context += "\n"

In [19]:
print(ontology_context)

Corticotropin is subclass of Diagnostic Procedure
Person
time:Instant
Amyotrophic Lateral Sclerosis Functional Rating Scale (ALSFRS) is subclass of Questionnaire
Activity
Administration
Adverse Drug Reaction
Anatomic Site
Before Onset is subclass of Event
CO concentrationCSF Analysis is subclass of Diagnostic Procedure
Clinical Evaluation is subclass of Diagnostic Procedure
Clinical Trial
Clinical Trial Participation
Clinics and Hospitals
Comorbidity is subclass of Conditon
Conditon
Diagnosis is subclass of Event
Diagnostic Imaging is subclass of Diagnostic Procedure
Diagnostic Procedure is subclass of Intervention or Procedure
Disease, Disorder or Finding
Expanded Disability Status Scale (EDSS) is subclass of Diagnostic Procedure
End Therapy Reason
Event
Evoked Potentials is subclass of Diagnostic Procedure
Gene
Genetic Testing
Group (social concept)
Hematology Test is subclass of Diagnostic Procedure
Impairment is subclass of Conditon
Intervention or Procedure
King's Clinical Staging

In [29]:
from llama_cpp_agent.llm_output_settings import LlmStructuredOutputSettings
from llama_cpp_agent.llm_agent import LlamaCppAgent
from llama_cpp_agent import FunctionCallingAgent, LlamaCppFunctionTool

from llama_cpp_agent.chat_history import (
    BasicChatHistory,
    BasicChatMessageStore,
    BasicChatHistoryStrategy,
)
from llama_cpp_agent.messages_formatter import MessagesFormatterType
import math

from typing import Union

chosen_classes = []


def suggest_classes(classes: list[str]):
    """
    Suggest classes to the user.

    Args:
        classes: List of classes to suggest to the user.

    """
    global chosen_classes
    chosen_classes = classes
    return "classes suggested"


def create_linked_classes(class_a: str, class_b: str):
    """
    Create a link between two classes.

    Args:
        class_a: First class to show in the visualization.
        class_b: Linked class to show.
    """
    global chosen_classes
    chosen_classes.append(class_a)
    chosen_classes.append(class_b)
    return "classes linked"


# Now let's create an instance of the LlmStructuredOutput class by calling the `from_functions` function of it and passing it a list of functions.

output_settings = LlmStructuredOutputSettings.from_functions(
    [create_linked_classes], allow_parallel_function_calling=True
)
linked_class_tool= LlamaCppFunctionTool(create_linked_classes)


# chat_history = BasicChatHistory(
#     chat_message_store=BasicChatMessageStore(),
#     chat_history_strategy=BasicChatHistoryStrategy(),
# )

# Create a LlamaCppAgent instance as before, including a system message with information about the tools available for the LLM agent.
llama_cpp_agent = LlamaCppAgent(
    provider,
    debug_output=True,
    system_prompt=f"You are an advanced AI, working with an ontology consisting of the following classes\n{ontology_context}\n You are tasked to assist the user by calling functions in JSON format.",
    predefined_messages_formatter_type=MessagesFormatterType.CHATML,
)

# Define some user input
user_input = "Recommend 2 different linked classes to me. I am interested in the nervous system."

# Pass the user input together with output settings to `get_chat_response` method.
# This will print the result of the function the LLM will call, it is a list of dictionaries containing the result.
response = llama_cpp_agent.get_chat_response(
    user_input, structured_output_settings=output_settings
)


def send_message_to_user_callback(message: str):
    print(message)
function_call_agent = FunctionCallingAgent(
    provider,
    llama_cpp_function_tools=[linked_class_tool],
    send_message_to_user_callback=send_message_to_user_callback,
    allow_parallel_function_calling=True,
    messages_formatter_type=MessagesFormatterType.CHATML)

function_call_agent.generate_response(user_input)

<|im_start|>system
Read and follow the instructions below:

<system_instructions>
You are an advanced AI, working with an ontology consisting of the following classes
Corticotropin is subclass of Diagnostic Procedure
Person
time:Instant
Amyotrophic Lateral Sclerosis Functional Rating Scale (ALSFRS) is subclass of Questionnaire
Activity
Administration
Adverse Drug Reaction
Anatomic Site
Before Onset is subclass of Event
CO concentrationCSF Analysis is subclass of Diagnostic Procedure
Clinical Evaluation is subclass of Diagnostic Procedure
Clinical Trial
Clinical Trial Participation
Clinics and Hospitals
Comorbidity is subclass of Conditon
Conditon
Diagnosis is subclass of Event
Diagnostic Imaging is subclass of Diagnostic Procedure
Diagnostic Procedure is subclass of Intervention or Procedure
Disease, Disorder or Finding
Expanded Disability Status Scale (EDSS) is subclass of Diagnostic Procedure
End Therapy Reason
Event
Evoked Potentials is subclass of Diagnostic Procedure
Gene
Genetic 

Llama.generate: 631 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    4636.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   103 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5172.95 ms /   104 tokens
Llama.generate: 17 prefix-match hit, remaining 325 prompt tokens to eval



[
  {
    "function":  "create_linked_classes",
    "arguments": {
      "class_a": "Amyotrophic Lateral Sclerosis Functional Rating Scale (ALSFRS)",
      "class_b": "Expanded Disability Status Scale (EDSS)"
    }
  },
  {
    "function":  "create_linked_classes", 
    "arguments": {
      "class_a": "Evoked Potentials",
      "class_b": "Diagnostic Imaging"
    } 
  }
]


llama_perf_context_print:        load time =    4636.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   325 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   199 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   11006.11 ms /   524 tokens
Llama.generate: 541 prefix-match hit, remaining 42 prompt tokens to eval
llama_perf_context_print:        load time =    4636.12 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    42 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   121 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6368.42 ms /   163 tokens


Two sets of linked classes have been created for you: 1) Neuroscience and Neurology to explore the structure, function, and disorders related to the nervous system. 2) Psychology and Neuroscience to understand behavior and mental processes in relation to the nervous system.


[{'function': 'send_message',
  'arguments': {'content': 'Two sets of linked classes have been created for you: 1) Neuroscience and Neurology to explore the structure, function, and disorders related to the nervous system. 2) Psychology and Neuroscience to understand behavior and mental processes in relation to the nervous system.'},
  'return_value': 'Message sent.'}]

In [22]:
response[0]

{'function': 'create_linked_classes',
 'arguments': {'class_a': 'Clinical Evaluation',
  'class_b': 'Disease, Disorder or Finding'},
 'return_value': 'classes linked'}